In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas as pd

In [5]:
df = pd.read_csv('features/2003-12-18:2024-03-08.csv', index_col=['Date'], parse_dates=['Date'])

In [6]:
df.head()

,open,high,low,close,target,action,rolling_7_open,rolling_7_open_pct,rolling_7_high,rolling_7_high_pct,...,rolling_14_close,rolling_14_close_pct,month_avg_open,day_avg_open,month_avg_high,day_avg_high,month_avg_low,day_avg_low,month_avg_close,day_avg_close
Date,,,,,,,,,,,,,,,,,,,,,
2003-12-18,1.240202,1.244199,1.236598,1.242298,1.238497,0,1.227001,0.010759,1.233614,0.008580,...,1.221969,0.016636,1.218301,1.240202,1.224887,1.244199,1.214828,1.236598,1.221969,1.242298
2003-12-19,1.242205,1.243503,1.235102,1.238497,1.239895,1,1.229472,0.010357,1.236028,0.006047,...,1.224969,0.011044,1.219894,1.242205,1.226128,1.243503,1.216180,1.235102,1.223071,1.238497
2003-12-22,1.235697,1.244601,1.235697,1.239895,1.240003,1,1.231844,0.003128,1.239043,0.004486,...,1.227183,0.010358,1.220882,1.235697,1.227283,1.244601,1.217399,1.235697,1.224123,1.239895
2003-12-23,1.239895,1.267202,1.238804,1.240003,1.245299,1,1.234300,0.004533,1.244272,0.018429,...,1.229162,0.008819,1.222000,1.239895,1.229631,1.267202,1.218659,1.238804,1.225057,1.240003
2003-12-24,1.240095,1.247194,1.240003,1.245299,1.244803,0,1.237357,0.002213,1.246371,0.000660,...,1.231820,0.010943,1.223006,1.240095,1.230607,1.247194,1.219844,1.240003,1.226181,1.245299


In [7]:
# Pipeline for target Prediction (assuming binary a floating number)
target_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', GridSearchCV(RandomForestRegressor(),
                                param_grid={'n_estimators': [100, 200, 300],
                                            'max_depth': [5, 10, 15]}))
])


# Split data into features (X) and targets (y)
X_target = df.drop(columns=['target', 'action'])
y_target = df['target']

# Train pipelines
target_pipeline.fit(X_target, y_target)

In [ ]:

# Pipeline for Action Prediction (assuming binary buy/sell)
action_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', GridSearchCV(RandomForestClassifier(),
                                param_grid={'n_estimators': [100, 200, 300],
                                            'max_depth': [5, 10, 15]}))
])


# Split data into features (X) and targets (y)
X_action = df.drop(columns=['target', 'action'])
y_action = df['action']

# Train pipelines
action_pipeline.fit(X_action, y_action)
